#  Dynamic Volatility Surface & Options Strategy Simulator

## Comprehensive Demonstration Notebook

This notebook demonstrates a complete end-to-end workflow for:
1. **Fetching option chain data** from yfinance
2. **Calculating implied volatility** using Black-Scholes
3. **Building volatility surfaces** with multiple interpolation methods
4. **Visualizing** 2D smiles, term structure, and 3D surfaces
5. **Calculating Greeks** (Delta, Gamma, Vega, Theta, Rho)
6. **Implementing trading strategies** (Volatility Arbitrage & Gamma Scalping)
7. **Backtesting** with comprehensive performance metrics
8. **P&L attribution** analysis

---

**Author**: Your Name  
**Date**: November 2025  
**Ticker**: SPY (S&P 500 ETF)

In [ ]:
# Standard library imports
import os
import sys
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Add src to path
sys.path.append('..')

# Data manipulation
import numpy as np
import pandas as pd

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Our custom modules
from src.data import OptionDataFetcher, HistoricalVolatilityCalculator
from src.pricing import (
    black_scholes_call, black_scholes_put,
    ImpliedVolatilitySolver, calculate_implied_volatility_vectorized
)
from src.greeks import GreeksCalculator, calculate_greeks_dataframe
from src.surface import VolatilitySurface
from src.strategies import VolatilityArbitrageStrategy, GammaScalpingStrategy, DeltaHedger
from src.backtest import OptionsBacktester, PerformanceMetrics, generate_performance_report
from src.visualization import VolatilityVisualizer, StrategyVisualizer
from src.utils import format_percentage, format_currency

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.precision', 4)
plt.style.use('seaborn-v0_8-darkgrid')

print("✅ All imports successful!")
print(f"📅 Analysis Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

In [ ]:
# Clean the option chain
spy_clean = fetcher.clean_option_chain(
    spy_chain,
    min_volume=50,
    min_oi=100,
    max_bid_ask_spread=0.3
)

print(f"📊 After cleaning: {len(spy_clean):,} contracts ({len(spy_clean)/len(spy_chain):.1%} retained)")
print(f"\n Expiration dates available:")
expirations = sorted(spy_clean['expiry'].unique())
for i, exp in enumerate(expirations[:5], 1):
    days_to_exp = (exp - pd.Timestamp.now()).days
    print(f"  {i}. {exp.strftime('%Y-%m-%d')} ({days_to_exp} days)")

In [ ]:
# Initialize data fetcher
fetcher = OptionDataFetcher(cache_enabled=True, provider='yfinance')

# Fetch SPY option chain
print("📊 Fetching SPY option chain...")
spy_chain = fetcher.get_option_chain("SPY")

# Get spot price and other data
spot_price = fetcher.get_spot_price("SPY")
risk_free_rate = fetcher.get_risk_free_rate()
dividend_yield = fetcher.get_dividend_yield("SPY")

print(f"\n✅ Fetched {len(spy_chain):,} option contracts")
print(f"💰 SPY Spot Price: ${spot_price:.2f}")
print(f"📈 Risk-Free Rate: {risk_free_rate:.2%}")
print(f"💵 Dividend Yield: {dividend_yield:.2%}")
print(f"\nSample data:")
spy_chain.head(10)

## 2. Fetch Option Chain Data

Let's fetch real-time option chain data for SPY using yfinance.

## 1. Setup & Imports

First, let's import all necessary libraries and set up our environment.